# 순환 신경망(RNN)을 이용한 텍스트 분류

**학습 목표:**
- 순서가 중요한 시퀀스(sequence) 데이터 처리에 특화된 **순환 신경망(RNN)**의 원리를 이해합니다.
- RNN의 장기 의존성(long-term dependency) 문제를 해결한 **LSTM(Long Short-Term Memory)** 모델의 구조를 학습합니다.
- 텍스트 데이터를 신경망이 처리할 수 있도록 **단어 임베딩(Word Embedding)**과 **패딩(Padding)** 과정을 이해하고 적용합니다.
- **TensorFlow/Keras**를 사용하여 LSTM 모델을 구축하고, **IMDB 영화 리뷰 데이터셋**으로 긍정/부정 감성 분석을 수행합니다.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
import matplotlib.pyplot as plt

### (1) 하이퍼파라미터 설정

In [ ]:
vocab_size = 10000  # 어휘 사전의 크기 (자주 등장하는 단어 10,000개)
max_len = 256      # 각 리뷰의 최대 길이 (단어 수)
embedding_dim = 128 # 단어를 표현할 임베딩 벡터의 차원
batch_size = 64
epochs = 5

### (2) 데이터 준비: IMDB 영화 리뷰
IMDB 데이터셋은 이미 전처리되어 각 단어가 고유한 정수 인덱스로 변환되어 있습니다. 신경망에 입력하기 위해서는 모든 시퀀스의 길이를 동일하게 맞춰야 하므로, `pad_sequences`를 사용하여 길이를 `max_len`으로 통일합니다. 길이가 짧은 리뷰는 0으로 채우고(padding), 긴 리뷰는 잘라냅니다.

In [ ]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

X_train_padded = pad_sequences(X_train, maxlen=max_len, padding='post', truncating='post')
X_test_padded = pad_sequences(X_test, maxlen=max_len, padding='post', truncating='post')

print(f"Original sequence example (length {len(X_train[0])}):\n{X_train[0]}")
print(f"\nPadded sequence example (length {len(X_train_padded[0])}):\n{X_train_padded[0]}")

### (3) 모델 정의: 양방향 LSTM (Bidirectional LSTM)
- **Embedding Layer**: 정수 인덱스로 표현된 단어를 `embedding_dim` 차원의 밀집 벡터(dense vector)로 변환합니다. 이 과정에서 단어 간의 의미적 유사성을 학습합니다.
- **Bidirectional(LSTM)**: 텍스트를 앞에서 뒤로, 그리고 뒤에서 앞으로 양방향으로 처리하여 문맥 정보를 더 풍부하게 학습합니다.
- **Dropout**: 훈련 중 일부 뉴런을 무작위로 비활성화하여 과적합을 방지합니다.
- **Dense (sigmoid)**: 최종적으로 리뷰가 긍정(1)일 확률을 출력합니다.

In [ ]:
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_len),
    Bidirectional(LSTM(64, return_sequences=True)), # 다음 LSTM 층으로 시퀀스를 전달하기 위해 True
    Bidirectional(LSTM(32)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.summary()

### (4) 모델 컴파일 및 훈련
긍정/부정의 이진 분류 문제이므로 손실 함수로 `binary_crossentropy`를 사용합니다.

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train_padded, y_train, 
                      batch_size=batch_size, 
                      epochs=epochs, 
                      validation_data=(X_test_padded, y_test))

### (5) 모델 평가 및 학습 과정 시각화

In [ ]:
test_loss, test_acc = model.evaluate(X_test_padded, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc:.4f}')

# 학습 과정 시각화
history_df = pd.DataFrame(history.history)
history_df.plot(figsize=(12, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.title('Training and Validation Metrics')
plt.show()